In [1]:
%run classes_definitions.ipynb

#### Package of preprocessing functions

The idea of this notebook is to store the "common functions" that could be used by any data scientist later on to test or develop news directions (in term of algorithms) with those data at hand.

In [2]:
#package importation 
import pandas as pd
import json as js
import datetime
import numpy as np
from time import gmtime, strftime
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

Importation of all kind package that will come handy for the rest of functions

In [3]:
#percentage of the missing values
def percent_missing_values(dataframe):
    missing_values_columns = dataframe.apply(lambda x : sum(x.isnull().values) , axis = 0)
    missing_values_percent = missing_values_columns / dataframe.shape[0]
    return missing_values_percent

 "percent_missing_values" is a function designed to quickly see which percentage of values is missing in each column of the dataset 

In [4]:
def timestampToDate(timestamp):
    return strftime("%m-%d-%Y %H:%M", gmtime(float(timestamp)))

"timestampToDate()" is a function made to convert a timestamp integer into a date and hour if associated (ex : 1512147643-> '12-01-2017 12:01')

In [6]:
#binarisation of the target column
def binary_coding(column , modality):
     #extraction of the modalities for the target variable
    #binarisation
    binary_column = pd.get_dummies(column) 
    return binary_column[modality] 


"binary_coding" is a function that transform a two-modalities column in a dataframe into binary number column (ex : 'false'-> 0 , 'true'->1)

In [5]:
def toInt(s):
    try:
        res = int(s)
        if type(res) == int:
            return res
    except:
        return

def parse_int(liste):
    try:
        return list(map(lambda x : toInt(x) , liste))
    except:
        return
    

In [7]:
def occurency(item_liste):
    liste_occurency = [[x,item_liste.count(x)] for x in set(item_liste)]
    return sorted(liste_occurency)

"occurency" is a function created to measure the frequency of occuring for any specific item in a list (ex: occureny([1,1,2])->[1,2],[2,1])

In [8]:
import pandas as pd
user_segment = pd.read_excel("user_segments.xls" , header = 0)

Importation of the excel file user_segment under the form of a dataframe

In [9]:
#extraction des elements du json file colonne par colonne

def campaign_impressions(filename , segment_name):
    
    new_line = []
    with open(filename) as campaign_impressions_file:
        for line in campaign_impressions_file:
            json_content = js.loads(line)
            user_id = Impression(json_content).uuid
            timestamp = timestampToDate(Impression(json_content).timestamp)
        
            ad_campaign_id = Impression(json_content).ad_campaign
            content_id = Impression(json_content).content
            ad_campaign_imp_id = Impression(json_content).ad_campaign_imp
        
            os_user_device_id = Impression(json_content).os_user_device
            source_data_id = Impression(json_content).source_data
            app_ad_imp_id = Impression(json_content).app_ad_imp
        
            segment = Impression(json_content).userHasSegment(segment_name , user_segment)
            conversion = Impression(json_content).didConvert()
        
            new_line.append([user_id  ,timestamp , ad_campaign_id , content_id , ad_campaign_imp_id , 
                        os_user_device_id , source_data_id , app_ad_imp_id , segment , conversion])
            
    
    data = [[toInt(item[0]) , item[1]  , toInt(item[2]) , toInt(item[3]) , toInt(item[4]) , toInt(item[5]) , toInt(item[6])  , toInt(item[7]) , item[8] , item[9]]    for item in new_line]

    colname =  ['user_id' , 'timestamp' , 'ad_campaign_id' , 'content_id' , 'ad_campaign_imp_id' , 'os_user_device_id' , 'source_data_id' , 'app_ad_imp_id' ,segment_name.lower() , 'conversion']
    dbase = pd.DataFrame(data)
    dbase.columns=colname
    
    dbase = dbase.dropna()
    
    return dbase

    

"campaign_impressions" is a function that has been designed in order to store every information in the json file as a dataframe , but it does more in a sense that it crossed the user_segment dataframe and the json file in order to know to which segment  every user that has been exposed to the ad campaign